In [1]:
import sympy
from sympy import *
from sympy.vector import *
from sympy import symbols
D = Derivative

In [2]:
init_session()

IPython console for SymPy 1.13.3 (Python 3.13.3-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.13.3/



In [3]:
sympy.sqrt(8)

2⋅√2

In [4]:
x, y = symbols('x y')
expr = x + 2 * y
expr

x + 2⋅y

In [5]:
diff(cos(x) * exp(x), x)

   x           x       
- ℯ ⋅sin(x) + ℯ ⋅cos(x)

In [6]:
u = Function('u')(x, t)
v = Function('v')(x, t)
laplacian = Symbol('Δ')
phi = Function('φ')(x)
f = Function('f')(x, t)
c = symbols('c')

wave_eq = diff(u, t, 2) - c**2 * laplacian * u - f
print("Base equation: ")
Eq(wave_eq, 0)

Base equation: 


                            2              
   2                       ∂               
- c ⋅Δ⋅u(x, t) - f(x, t) + ───(u(x, t)) = 0
                             2             
                           ∂t              

Réarrangons les termes, avec les fonctions inconnues à gauche et les connues à droite.

In [7]:
terms = Add.make_args(wave_eq)

terms_with_u = [term for term in terms if term.has(u) or term.has(laplacian)]
terms_without_u = [term for term in terms if not (term.has(u) or term.has(laplacian))]

lhs = Add(*terms_with_u)
rhs = -Add(*terms_without_u)
Eq(lhs, rhs)

                  2                    
   2             ∂                     
- c ⋅Δ⋅u(x, t) + ───(u(x, t)) = f(x, t)
                   2                   
                 ∂t                    

Introduisont la variable $v = \frac{\partial u}{\partial t}$ afin de remplacer les dérivées secondes en temps en dérivée premières en temps.

Cela permettra de simplifier l'écriture du schéma.

In [8]:
lhs = lhs.subs(diff(u, t, 2), diff(v, t))
v_eq = diff(u, t) - v
[Eq(lhs, rhs), Eq(v_eq, 0)]

⎡   2             ∂                                 ∂              ⎤
⎢- c ⋅Δ⋅u(x, t) + ──(v(x, t)) = f(x, t), -v(x, t) + ──(u(x, t)) = 0⎥
⎣                 ∂t                                ∂t             ⎦

In [9]:
order = ode_order(lhs, v)
order

1

In [10]:
var_eq = Eq(Integral(lhs * v, x), Integral(rhs * v,x))
var_eq

⌠                                                                 
⎮ ⎛   2             ∂          ⎞              ⌠                   
⎮ ⎜- c ⋅Δ⋅u(x, t) + ──(v(x, t))⎟⋅v(x, t) dx = ⎮ f(x, t)⋅v(x, t) dx
⎮ ⎝                 ∂t         ⎠              ⌡                   
⌡                                                                 

### Time discretization

In [11]:
theta = symbols('theta')
def time_discr_fns(name: str):
    return map(lambda s: s(x), symbols(f'{name}^n {name}^n-1', cls=Function, commutative=True))
t_curr, t_prev = time_discr_fns('t')
u_curr, u_prev = time_discr_fns('u')
v_curr, v_prev = time_discr_fns('v')
f_curr, f_prev = time_discr_fns('f')

k = symbols('k')# t_curr - t_prev # length of current time step
exprs = (lhs, rhs, v_eq)
res = [expr.subs(
    diff(u, t),
    (u_curr - u_prev) / k
).subs(
    diff(v, t),
    (v_curr - v_prev) / k
)
.subs(
    f,
    theta * f_curr + (1 - theta) * f_prev
)
.subs(
    v,
    theta * v_curr + (1 - theta) * v_prev
)
.subs(
    u,
    theta * u_curr + (1 - theta) * u_prev
)
       for expr in exprs]
# res = [expr.subs(theta, 0) for expr in res]
lhs_d, rhs_d, v_eq_d = res

Première équation discretisée :

In [12]:
eq_1_d = Eq(v_eq_d, 0)
eq_1_d

                             uⁿ(x) - uⁿ⁻¹(x)    
-θ⋅vⁿ(x) - (1 - θ)⋅vⁿ⁻¹(x) + ─────────────── = 0
                                    k           

Seconde équation discretisée :

In [13]:
eq_2_d = Eq(lhs_d, rhs_d)
eq_2_d

   2                                 vⁿ(x) - vⁿ⁻¹(x)                            
- c ⋅Δ⋅(θ⋅uⁿ(x) + (1 - θ)⋅uⁿ⁻¹(x)) + ─────────────── = θ⋅fⁿ(x) + (1 - θ)⋅fⁿ⁻¹(x)
                                            k                                   

**Crank-Nicolson** ($\theta = \frac{1}{2}$)

In [14]:
eq_1_d.subs(theta, Rational(1, 2))

  vⁿ(x)   vⁿ⁻¹(x)   uⁿ(x) - uⁿ⁻¹(x)    
- ───── - ─────── + ─────────────── = 0
    2        2             k           

In [15]:
def mult_eq(eq, expr):
    return Eq(eq.lhs * expr, eq.rhs * expr)

a = eq_1_d.subs(v_curr, solve(eq_2_d, v_curr)[0])
eq_1_d_s = Eq(u_curr, solve(a, u_curr)[0])
eq_1_d_s = mult_eq(eq_1_d_s, denom(eq_1_d_s.rhs))
eq_1_d_s

⎛ 2  2  2      ⎞          2  2  2              2  2                2  2        ↪
⎝c ⋅k ⋅θ ⋅Δ - 1⎠⋅uⁿ(x) = c ⋅k ⋅θ ⋅Δ⋅uⁿ⁻¹(x) - c ⋅k ⋅θ⋅Δ⋅uⁿ⁻¹(x) - k ⋅θ ⋅fⁿ(x)  ↪

↪    2  2            2                                
↪ + k ⋅θ ⋅fⁿ⁻¹(x) - k ⋅θ⋅fⁿ⁻¹(x) - k⋅vⁿ⁻¹(x) - uⁿ⁻¹(x)

In [16]:
eq_2_d_s = Eq(v_curr, solve(eq_2_d, v_curr)[0])
eq_2_d_s

         2                2                  2                                 ↪
vⁿ(x) = c ⋅k⋅θ⋅Δ⋅uⁿ(x) - c ⋅k⋅θ⋅Δ⋅uⁿ⁻¹(x) + c ⋅k⋅Δ⋅uⁿ⁻¹(x) + k⋅θ⋅fⁿ(x) - k⋅θ⋅f ↪

↪                             
↪ ⁿ⁻¹(x) + k⋅fⁿ⁻¹(x) + vⁿ⁻¹(x)

In [17]:


def integ_eq(eq):
    return Eq(Integral(eq.lhs, x), Integral(eq.rhs, x))

def variational(eq):
    return integ_eq(mult_eq(eq, phi))

eq_1_v = variational(eq_1_d_s)
a = expand(factor(expand(eq_1_v), deep=True))
a

 2  2  2   ⌠                 ⌠                  2  2  2   ⌠                    ↪
c ⋅k ⋅θ ⋅Δ⋅⎮ uⁿ(x)⋅φ(x) dx - ⎮ uⁿ(x)⋅φ(x) dx = c ⋅k ⋅θ ⋅Δ⋅⎮ uⁿ⁻¹(x)⋅φ(x) dx -  ↪
           ⌡                 ⌡                            ⌡                    ↪

↪  2  2     ⌠                    2  2 ⌠                  2  2 ⌠                ↪
↪ c ⋅k ⋅θ⋅Δ⋅⎮ uⁿ⁻¹(x)⋅φ(x) dx - k ⋅θ ⋅⎮ fⁿ(x)⋅φ(x) dx + k ⋅θ ⋅⎮ fⁿ⁻¹(x)⋅φ(x) d ↪
↪           ⌡                         ⌡                       ⌡                ↪

↪      2   ⌠                     ⌠                   ⌠                
↪ x - k ⋅θ⋅⎮ fⁿ⁻¹(x)⋅φ(x) dx - k⋅⎮ vⁿ⁻¹(x)⋅φ(x) dx - ⎮ uⁿ⁻¹(x)⋅φ(x) dx
↪          ⌡                     ⌡                   ⌡                

In [18]:
nabla = symbols('∇', cls=Function)
def decompose_integrals(expr):
    args = Add.make_args(expr)
    res = {}

    for arg in args:
        if isinstance(arg, Integral):
            res.setdefault(arg, []).append(1)
        elif isinstance(arg, Mul):
            factors = Mul.make_args(arg)
            integral = list(filter(lambda e: isinstance(e, Integral), factors))
            coeff = Mul(*filter(lambda e: not isinstance(e, Integral), factors))
            assert len(integral) == 1
            integral = integral[0]

            if coeff.has(laplacian):
                # Integrate by parts to reduce spatial derivatives order
                coeff /= laplacian
                coeff = -coeff
                
                fns, dx = integral.args
                f, shape = Mul.make_args(fns)
                integral = Integral(nabla(f) * nabla(shape), dx)
            
            res.setdefault(integral, []).append(coeff)
            

        else:
            raise Exception("Help")

    for key, coeffs in res.items():
        res[key] = factor(Add(*coeffs), deep=True)
        
    return res


a = expand(factor(expand(variational(eq_1_d_s)), deep=True))
a
dec = decompose_integrals(a.rhs)
dec

⎧⌠                  2  2  ⌠                   2            ⌠                   ↪
⎨⎮ fⁿ(x)⋅φ(x) dx: -k ⋅θ , ⎮ fⁿ⁻¹(x)⋅φ(x) dx: k ⋅θ⋅(θ - 1), ⎮ uⁿ⁻¹(x)⋅φ(x) dx:  ↪
⎩⌡                        ⌡                                ⌡                   ↪

↪     ⌠                      ⌠                          2  2          ⎫
↪ -1, ⎮ vⁿ⁻¹(x)⋅φ(x) dx: -k, ⎮ ∇(uⁿ⁻¹(x))⋅∇(φ(x)) dx: -c ⋅k ⋅θ⋅(θ - 1)⎬
↪     ⌡                      ⌡                                        ⎭

In [19]:
def compose_integrals(decomposition: dict[Integral, Expr]):
    return Add(*(Mul(coeff, integral) for integral, coeff in decomposition.items()))

compose_integrals(dec)

   2  2           ⌠                          2  2 ⌠                  2         ↪
- c ⋅k ⋅θ⋅(θ - 1)⋅⎮ ∇(uⁿ⁻¹(x))⋅∇(φ(x)) dx - k ⋅θ ⋅⎮ fⁿ(x)⋅φ(x) dx + k ⋅θ⋅(θ -  ↪
                  ⌡                               ⌡                            ↪

↪    ⌠                     ⌠                   ⌠                
↪ 1)⋅⎮ fⁿ⁻¹(x)⋅φ(x) dx - k⋅⎮ vⁿ⁻¹(x)⋅φ(x) dx - ⎮ uⁿ⁻¹(x)⋅φ(x) dx
↪    ⌡                     ⌡                   ⌡                

In [20]:
def simplify_integrals(expr):
    return compose_integrals(decompose_integrals(expr))

In [21]:
def simplify_variational(eq):
    eq = expand(factor(expand(eq), deep=True))
    return Eq(simplify_integrals(eq.lhs), simplify_integrals(eq.rhs))

def simplified_variational(eq):
    return simplify_variational(variational(eq))

eq_1_v = simplified_variational(eq_1_d_s)
eq_1_v

   2  2  2 ⌠                       ⌠                    2  2           ⌠       ↪
- c ⋅k ⋅θ ⋅⎮ ∇(uⁿ(x))⋅∇(φ(x)) dx - ⎮ uⁿ(x)⋅φ(x) dx = - c ⋅k ⋅θ⋅(θ - 1)⋅⎮ ∇(uⁿ⁻ ↪
           ⌡                       ⌡                                   ⌡       ↪

↪                     2  2 ⌠                  2           ⌠                    ↪
↪ ¹(x))⋅∇(φ(x)) dx - k ⋅θ ⋅⎮ fⁿ(x)⋅φ(x) dx + k ⋅θ⋅(θ - 1)⋅⎮ fⁿ⁻¹(x)⋅φ(x) dx -  ↪
↪                          ⌡                              ⌡                    ↪

↪   ⌠                   ⌠                
↪ k⋅⎮ vⁿ⁻¹(x)⋅φ(x) dx - ⎮ uⁿ⁻¹(x)⋅φ(x) dx
↪   ⌡                   ⌡                

In [22]:
eq_2_v = simplified_variational(eq_2_d_s)
eq_2_v

⌠                    2     ⌠                        2           ⌠              ↪
⎮ vⁿ(x)⋅φ(x) dx = - c ⋅k⋅θ⋅⎮ ∇(uⁿ(x))⋅∇(φ(x)) dx + c ⋅k⋅(θ - 1)⋅⎮ ∇(uⁿ⁻¹(x))⋅∇ ↪
⌡                          ⌡                                    ⌡              ↪

↪                 ⌠                           ⌠                   ⌠            ↪
↪ (φ(x)) dx + k⋅θ⋅⎮ fⁿ(x)⋅φ(x) dx - k⋅(θ - 1)⋅⎮ fⁿ⁻¹(x)⋅φ(x) dx + ⎮ vⁿ⁻¹(x)⋅φ( ↪
↪                 ⌡                           ⌡                   ⌡            ↪

↪      
↪ x) dx
↪      

In [23]:
i, h, U_i_n = symbols("i h U^n_i")
shape_i_n = Function("phi^n_i")(x)
Eq(u_curr, Sum(U_i_n * shape_i_n, (i, 0, h)) + theta * Sum(U_i_n * shape_i_n, (i, 0, h)))

            h                  h             
           ___                ___            
           ╲                  ╲              
            ╲                  ╲             
uⁿ(x) = θ⋅  ╱   Uⁿᵢ⋅φⁿᵢ(x) +   ╱   Uⁿᵢ⋅φⁿᵢ(x)
           ╱                  ╱              
           ‾‾‾                ‾‾‾            
          i = 0              i = 0           

In [24]:
from typing import Iterable

def approximated_unknown(unknown: Function):
    name = str(unknown.func).split("_")[0]
    i, h = symbols(f"i h")
    coeff = Function(f"{name.upper()}^n_i")(i, x)
    shape_i_n = Function("phi^n_i")(i, x)
    return Sum(coeff * shape_i_n, (i, 0, h))

def approximate_unknowns(eq: Expr, unknowns: Iterable[Function]):
    substitutions = {unknown: approximated_unknown(unknown) for unknown in unknowns}
    return eq.subs(substitutions, simultaneous=True)

time_discr_unknowns = (u_curr, u_prev, v_curr, v_prev)
eq_1_f = approximate_unknowns(eq_1_v, time_discr_unknowns)
eq_1_f

           ⌠                                                 ⌠                 ↪
           ⎮          ⎛  h                            ⎞      ⎮        h        ↪
           ⎮          ⎜ ___                           ⎟      ⎮       ___       ↪
           ⎮          ⎜ ╲                             ⎟      ⎮       ╲         ↪
   2  2  2 ⎮          ⎜  ╲                            ⎟      ⎮        ╲        ↪
- c ⋅k ⋅θ ⋅⎮ ∇(φ(x))⋅∇⎜  ╱   U_i__N__n(i, x)⋅φⁿᵢ(i, x)⎟ dx - ⎮ φ(x)⋅  ╱   U_i_ ↪
           ⎮          ⎜ ╱                             ⎟      ⎮       ╱         ↪
           ⎮          ⎜ ‾‾‾                           ⎟      ⎮       ‾‾‾       ↪
           ⎮          ⎝i = 0                          ⎠      ⎮      i = 0      ↪
           ⌡                                                 ⌡                 ↪

↪                                              ⌠                               ↪
↪                                              ⎮          ⎛  h                 ↪
↪                          

In [25]:
def simplify_integral_of_sum(expr: Integral):
    
    return expr

simplify_integral_of_sum(Integral(phi * approximated_unknown(u_curr), x))

⌠                                        
⎮        h                               
⎮       ___                              
⎮       ╲                                
⎮        ╲                               
⎮ φ(x)⋅  ╱   U_i__N__n(i, x)⋅φⁿᵢ(i, x) dx
⎮       ╱                                
⎮       ‾‾‾                              
⎮      i = 0                             
⌡                                        

In [26]:
nabla = Function("∇")
simplify(nabla(approximated_unknown(u_curr)))

 ⎛  h                            ⎞
 ⎜ ___                           ⎟
 ⎜ ╲                             ⎟
 ⎜  ╲                            ⎟
∇⎜  ╱   U_i__N__n(i, x)⋅φⁿᵢ(i, x)⎟
 ⎜ ╱                             ⎟
 ⎜ ‾‾‾                           ⎟
 ⎝i = 0                          ⎠

In [27]:
eq_1_v

   2  2  2 ⌠                       ⌠                    2  2           ⌠       ↪
- c ⋅k ⋅θ ⋅⎮ ∇(uⁿ(x))⋅∇(φ(x)) dx - ⎮ uⁿ(x)⋅φ(x) dx = - c ⋅k ⋅θ⋅(θ - 1)⋅⎮ ∇(uⁿ⁻ ↪
           ⌡                       ⌡                                   ⌡       ↪

↪                     2  2 ⌠                  2           ⌠                    ↪
↪ ¹(x))⋅∇(φ(x)) dx - k ⋅θ ⋅⎮ fⁿ(x)⋅φ(x) dx + k ⋅θ⋅(θ - 1)⋅⎮ fⁿ⁻¹(x)⋅φ(x) dx -  ↪
↪                          ⌡                              ⌡                    ↪

↪   ⌠                   ⌠                
↪ k⋅⎮ vⁿ⁻¹(x)⋅φ(x) dx - ⎮ uⁿ⁻¹(x)⋅φ(x) dx
↪   ⌡                   ⌡                

In [28]:
eq_2_v

⌠                    2     ⌠                        2           ⌠              ↪
⎮ vⁿ(x)⋅φ(x) dx = - c ⋅k⋅θ⋅⎮ ∇(uⁿ(x))⋅∇(φ(x)) dx + c ⋅k⋅(θ - 1)⋅⎮ ∇(uⁿ⁻¹(x))⋅∇ ↪
⌡                          ⌡                                    ⌡              ↪

↪                 ⌠                           ⌠                   ⌠            ↪
↪ (φ(x)) dx + k⋅θ⋅⎮ fⁿ(x)⋅φ(x) dx - k⋅(θ - 1)⋅⎮ fⁿ⁻¹(x)⋅φ(x) dx + ⎮ vⁿ⁻¹(x)⋅φ( ↪
↪                 ⌡                           ⌡                   ⌡            ↪

↪      
↪ x) dx
↪      

In [81]:
shape_i_curr, shape_i_prev, shape_j_curr, shape_j_prev = map(lambda f: f(x), symbols('phi_i^n phi_i^n-1 phi_j^n phi_j^n-1', cls=Function))



def space_discretize_integral(integral: Integral, unknowns: Iterable[Function], knowns: Iterable[Function]):
    product, dx = integral.args
    l, r = Mul.make_args(product)
    assert r.has(phi), "The shape function should be the right term."

    

    unknowns_present = tuple(filter(lambda unknown: l.has(unknown), unknowns))
    assert len(unknowns_present) <= 1, "At most one of the time discretized functions should be present."

    if len(unknowns_present) == 0:
        knowns_present = tuple(filter(lambda known: l.has(known), knowns))
        assert len(knowns_present) == 1, "One of the time discretized known functions should be present."
        fn = knowns_present[0]
        name, time_step = str(fn.func).split('^')
        return Integral(l * Function(f'phi_j^{time_step}')(x))

    fn = unknowns_present[0]
    name, time_step = str(fn.func).split('^')

    
    
    return Sum(Symbol(f'{name.upper()}_i^{time_step}') * Integral(l.subs(fn, Function(f'phi_i^n')(x)) * r.subs(phi, Function(f'phi_j^{time_step}')(x)), x), (i, 0, h))

time_discr_knowns = (f_curr, f_prev)
time_discr_fns = time_discr_unknowns + (f_curr, f_prev)
integral =Integral(nabla(v_curr) * nabla(phi), x)
res = space_discretize_integral(integral, time_discr_unknowns, time_discr_knowns)
Eq(integral, res)

                         h                                   
                        ____                                 
                        ╲                                    
                         ╲                                   
⌠                         ╲       ⌠                          
⎮ ∇(vⁿ(x))⋅∇(φ(x)) dx =   ╱   Vⁿᵢ⋅⎮ ∇(φⁿᵢ(x))⋅∇(φ_j__n(x)) dx
⌡                        ╱        ⌡                          
                        ╱                                    
                        ‾‾‾‾                                 
                        i = 0                                

In [44]:
def space_discretize(expr: Expr):
    if isinstance(expr, Symbol) or isinstance(expr, Number):
        return expr
    elif not isinstance(expr, Integral):
        return expr.func(*map(space_discretize, expr.args))
    else:
        return space_discretize_integral(expr, time_discr_unknowns, time_discr_knowns)

eq_1_f = space_discretize(eq_1_v)
eq_1_f

            h                                       h                          ↪
           ____                                    ____                        ↪
           ╲                                       ╲                           ↪
            ╲                                       ╲                          ↪
   2  2  2   ╲       ⌠                               ╲       ⌠                 ↪
- c ⋅k ⋅θ ⋅  ╱   Uⁿᵢ⋅⎮ ∇(φⁿᵢ(x))⋅∇(φ_j__n(x)) dx -   ╱   Uⁿᵢ⋅⎮ φⁿᵢ(x)⋅φ_j__n(x ↪
            ╱        ⌡                              ╱        ⌡                 ↪
           ╱                                       ╱                           ↪
           ‾‾‾‾                                    ‾‾‾‾                        ↪
           i = 0                                   i = 0                       ↪

↪                           h                                                  ↪
↪                          ____                                                ↪
↪                          

In [45]:
eq_2_f = space_discretize(eq_2_v)
eq_2_f

 h                                          h                                  ↪
____                                       ____                                ↪
╲                                          ╲                                   ↪
 ╲                                          ╲                                  ↪
  ╲       ⌠                          2       ╲       ⌠                         ↪
  ╱   Vⁿᵢ⋅⎮ φⁿᵢ(x)⋅φ_j__n(x) dx = - c ⋅k⋅θ⋅  ╱   Uⁿᵢ⋅⎮ ∇(φⁿᵢ(x))⋅∇(φ_j__n(x))  ↪
 ╱        ⌡                                 ╱        ⌡                         ↪
╱                                          ╱                                   ↪
‾‾‾‾                                       ‾‾‾‾                                ↪
i = 0                                      i = 0                               ↪

↪                    h                                                         ↪
↪                   ____                                                       ↪
↪                   ╲      

#### Matrixify Integral

In [116]:
def NCSymbol(*args, **kwargs):
    return Symbol(*args, commutative=False, **kwargs)

def matrixify_integral(integral: Integral):
    assert isinstance(integral, Integral)

    l, r = Mul.make_args(integral.args[0])

    if l == shape_i_curr and r == shape_j_curr:
        return NCSymbol('M^n')
    if l == shape_i_curr and r == shape_j_prev:
        return NCSymbol('M^n,n-1')
    elif l == nabla(shape_i_curr) and r == nabla(shape_j_curr):
        return NCSymbol('A^n')
    elif l == nabla(shape_i_curr) and r == nabla(shape_j_prev):
        return NCSymbol('A^n,n-1')
    elif r == shape_j_curr:
        assert isinstance(l, Function)
        name = str(l.func).split("^")[0]
        return NCSymbol(f'{name.upper()}^n')
    elif r == shape_j_prev:
        assert isinstance(l, Function)
        name = str(l.func).split("^")[0]
        return NCSymbol(f'{name.upper()}^n-1')
        

    
    raise Exception(f"Unknown integral to matrixify: {integral}")

integral = Integral(f_curr * shape_j_prev, x)
(integral, matrixify_integral(integral))

⎛⌠                           ⎞
⎜⎮ fⁿ(x)⋅φ_j__n-1(x) dx, Fⁿ⁻¹⎟
⎝⌡                           ⎠

In [ ]:
integral = Integral(nabla(shape_i_curr) * nabla(shape_j_curr), x)
(integral, matrixify_integral(integral))

In [105]:
integral = Integral(shape_i_curr * shape_j_prev, x)
(integral, matrixify_integral(integral))

⎛⌠                                ⎞
⎜⎮ φⁿᵢ(x)⋅φ_j__n-1(x) dx, M__n,n-1⎟
⎝⌡                                ⎠

In [117]:
def matrixify_sum(expr: Sum):
    assert isinstance(expr, Sum), "Expression should be a sum."
    coeff, integral = Mul.make_args(expr.args[0])
    assert isinstance(coeff, Symbol)
    assert isinstance(integral, Integral)
    fns = Mul.make_args(integral.args[0])

    coeff = str(coeff)
    name = coeff.split('_')[0]
    time_step = coeff.split('^')[1]
    return matrixify_integral(integral) * Symbol(str(coeff).replace('_i', '').replace('_j', ''), commutative=False)

integral = Integral(nabla(v_curr) * nabla(phi), x)
sum_ = space_discretize_integral(integral, time_discr_unknowns, time_discr_knowns)
sum_, matrixify_sum(sum_)

⎛ h                                          ⎞
⎜____                                        ⎟
⎜╲                                           ⎟
⎜ ╲                                          ⎟
⎜  ╲       ⌠                                 ⎟
⎜  ╱   Vⁿᵢ⋅⎮ ∇(φⁿᵢ(x))⋅∇(φ_j__n(x)) dx, Aⁿ⋅Vⁿ⎟
⎜ ╱        ⌡                                 ⎟
⎜╱                                           ⎟
⎜‾‾‾‾                                        ⎟
⎝i = 0                                       ⎠

In [119]:
(eq_2_f.lhs, matrixify_sum(eq_2_f.lhs))

⎛ h                                    ⎞
⎜____                                  ⎟
⎜╲                                     ⎟
⎜ ╲                                    ⎟
⎜  ╲       ⌠                           ⎟
⎜  ╱   Vⁿᵢ⋅⎮ φⁿᵢ(x)⋅φ_j__n(x) dx, Mⁿ⋅Vⁿ⎟
⎜ ╱        ⌡                           ⎟
⎜╱                                     ⎟
⎜‾‾‾‾                                  ⎟
⎝i = 0                                 ⎠

In [135]:
def matrixify(expr: Expr):
    if isinstance(expr, Symbol) or isinstance(expr, Number):
        return expr
    if isinstance(expr, Sum):
        return matrixify_sum(expr)
    elif isinstance(expr, Integral):
        return matrixify_integral(expr)
    else:
        return expr.func(*map(matrixify, expr.args))

eq_1_m = factor(matrixify(eq_1_f), deep=True)
eq_1_m

⎛   2  2  2        ⎞         2  2                            2  2       2      ↪
⎝- c ⋅k ⋅θ ⋅Aⁿ - Mⁿ⎠⋅Uⁿ = - c ⋅k ⋅θ⋅(θ - 1)⋅A__n,n-1⋅Uⁿ⁻¹ - k ⋅θ ⋅Fⁿ + k ⋅θ⋅(θ ↪

↪                                             
↪  - 1)⋅Fⁿ⁻¹ - k⋅M__n,n-1⋅Vⁿ⁻¹ - M__n,n-1⋅Uⁿ⁻¹

In [134]:
eq_2_m = factor(matrixify(eq_2_f), deep=True)
eq_2_m

           2              2                                                    ↪
Mⁿ⋅Vⁿ = - c ⋅k⋅θ⋅Aⁿ⋅Uⁿ + c ⋅k⋅(θ - 1)⋅A__n,n-1⋅Uⁿ⁻¹ + k⋅θ⋅Fⁿ - k⋅(θ - 1)⋅Fⁿ⁻¹  ↪

↪                
↪ + M__n,n-1⋅Vⁿ⁻¹